[data] #DAQO-415 - Tính xem có thể mua được tối đa bao nhiêu crypto với tỉ lệ trượt giá 1%

In [1]:
from sqlalchemy import create_engine
import re
import pandas as pd
import numpy as np
from datetime import datetime
import seaborn as sns
import matplotlib.pyplot as plt

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 50)
pd.options.display.float_format = "{:,.6f}".format

In [2]:
## setup url to connect redshift by postgres driver

server = ""
port = "5439"
database = "analytic"

username = 'metabase'
password = ''

connection_string = f'postgresql://{username}:{password}@{server}:{port}/{database}'
engine = create_engine(connection_string, pool_pre_ping=True)

In [3]:
query = "select * from amm_positions"
df_positions_all = pd.read_sql(sql=query, con=engine)
df_positions_all.head()

,user_id,id,ref,amm_pool_id,status,status_explanation,token0,amount0_desired,token1,amount1_desired,price_scale,starting_at,stopping_at,stopped_at,created_at,updated_at,amount0_initial,amount1_initial,amount0_principal,amount1_principal,amount0,amount1,fee0,fee1,liquidity,tick_lower,tick_upper,slippage_tolerance,in_range,status_explanation_details,user_status,doc_country,doc_status,phone_number_status,peer_certify_status,first_trade_at,level
0,969422,9905,AP7009971537,2,stopped,None,eth,0.002660,usdt,7.000000,0.100000,2022-01-07 04:21:02.026862,2022-01-08 04:52:43.533630,2022-01-08 04:52:48.238109,2022-01-07 04:21:01.977363,2022-01-08 04:52:48.466525,0.002660,6.999029,0.002433,7.724497,0.002438,7.742063,0.000005,0.017565,2.655872,79695,81801,0.010000,True,None,active,vn,verified,verified,peer_certify_pending,2018-05-23 06:29:35.170734,3
1,969422,10265,AP8897393369,2,stopped,None,eth,0.002653,usdt,8.540000,0.100000,2022-01-08 04:56:12.915037,2022-01-15 08:32:47.010547,2022-01-15 08:32:50.599954,2022-01-08 04:56:12.839949,2022-01-15 08:32:50.959517,0.002653,8.516366,0.001845,11.149142,0.001866,11.216962,0.000020,0.067819,2.925363,79690,81796,0.010000,True,None,active,vn,verified,verified,peer_certify_pending,2018-05-23 06:29:35.170734,3
2,969422,9716,AP9912088842,2,stopped,None,eth,0.003310,usdt,11.090000,1.000000,2022-01-06 10:58:58.147325,2022-01-07 04:19:40.002836,2022-01-07 04:19:42.307889,2022-01-06 10:58:58.083849,2022-01-07 04:19:42.590252,0.003310,11.090000,0.003380,10.858748,0.003380,10.859227,0.000000,0.000479,0.191604,-103044,265389,0.050000,True,None,active,vn,verified,verified,peer_certify_pending,2018-05-23 06:29:35.170734,3
3,969422,10271,AP1806045459,1,stopped,None,btc,0.000192,usdt,8.030000,0.100000,2022-01-08 05:31:29.235484,2022-01-16 07:11:30.630261,2022-01-16 07:11:32.512066,2022-01-08 05:31:29.168382,2022-01-16 07:11:32.734412,0.000191,8.030000,0.000140,10.208283,0.000142,10.314416,0.000002,0.106132,0.762242,105373,107479,0.010000,True,"amount1 < amount1Min, amount1=9874144100118363...",active,vn,verified,verified,peer_certify_pending,2018-05-23 06:29:35.170734,3
4,969422,10641,AP1791507820,100,stopped,None,ltc,0.040000,usdt,5.220000,0.100000,2022-01-10 06:56:46.453708,2022-01-14 15:38:03.749067,2022-01-14 15:38:04.790849,2022-01-10 06:56:46.409372,2022-01-14 15:38:05.048990,0.039855,5.220000,0.000000,10.706677,0.000146,10.731448,0.000146,0.024771,8.836470,47668,49774,0.050000,False,None,active,vn,verified,verified,peer_certify_pending,2018-05-23 06:29:35.170734,3


In [4]:
for col in df_positions_all.columns:
    if df_positions_all[col].dtype == 'float':
        df_positions_all[col] = df_positions_all[col].astype(np.float64)

# SET VARIABLES

In [5]:
TOKEN0 = 'usdt'
TOKEN1 = 'ngn'

# PRICE_TYPE = 0: price of token0 based on token1
# PRICE_TYPE = 1: price of token1 based on token0
PRICE_TYPE = 0

CURRENT_PRICE = 582
TARGET_SPLIPPAGE = 0.01 # 1%

DATETIME_REPORT= '2022-03-31 12:00:00'

# Calculation

In [6]:
TOKEN0 = TOKEN0.lower()
TOKEN1 = TOKEN1.lower()

df_positions = df_positions_all[(df_positions_all['token0'] == TOKEN0) 
                                & (df_positions_all['token1'] == TOKEN1)
                                & (df_positions_all['starting_at'] <= DATETIME_REPORT)
                                & ((df_positions_all['stopped_at'] >= DATETIME_REPORT) | df_positions_all['stopped_at'].isna())
                               ] \
                    .reset_index(drop=True)

print(f'Selected token 0: {TOKEN0}')
print(f'Selected token 1: {TOKEN1}')
print(f'Report date is at {DATETIME_REPORT}')
print(f'Total positions made in that pool: {len(df_positions):,}')
print(f'Total positions created in that pool: {len(df_positions[df_positions["status"] != "aborted"]):,}')

Selected token 0: usdt
Selected token 1: ngn
Report date is at 2022-03-31 12:00:00
Total positions made in that pool: 282
Total positions created in that pool: 49


In [7]:
df_positions.head()

,user_id,id,ref,amm_pool_id,status,status_explanation,token0,amount0_desired,token1,amount1_desired,price_scale,starting_at,stopping_at,stopped_at,created_at,updated_at,amount0_initial,amount1_initial,amount0_principal,amount1_principal,amount0,amount1,fee0,fee1,liquidity,tick_lower,tick_upper,slippage_tolerance,in_range,status_explanation_details,user_status,doc_country,doc_status,phone_number_status,peer_certify_status,first_trade_at,level
0,706486,3041,AP8139745730,67,aborted,slippage_tolerance_exceeded,usdt,"24,776.260000",ngn,"11,772,249.000000",nan,2021-12-10 15:36:24.083499,NaT,NaT,2021-12-10 15:36:23.818068,2021-12-10 15:36:27.769094,nan,nan,nan,nan,nan,nan,nan,nan,nan,63371,63484,0.050000,None,"amount1 < amount1Min, amount1=1118324554331227...",active,vn,verified,verified,admin_certified,2019-12-10 15:41:04.976560,4
1,706486,2146,AP9878477516,67,aborted,slippage_tolerance_exceeded,usdt,"9,946.730000",ngn,"5,606,399.000000",0.100000,2021-12-06 02:42:13.626794,NaT,NaT,2021-12-06 02:42:13.612888,2021-12-06 02:42:18.727878,nan,nan,nan,nan,nan,nan,nan,nan,nan,62293,64401,0.005000,None,"amount1 < amount1Min, amount1=5558746905644153...",active,vn,verified,verified,admin_certified,2019-12-10 15:41:04.976560,4
2,706486,3043,AP3327367743,67,aborted,slippage_tolerance_exceeded,usdt,"24,776.260000",ngn,"11,772,249.000000",nan,2021-12-10 15:38:18.081871,NaT,NaT,2021-12-10 15:38:17.833474,2021-12-10 15:38:20.097005,nan,nan,nan,nan,nan,nan,nan,nan,nan,63371,63484,0.050000,None,"amount1 < amount1Min, amount1=1118324554331227...",active,vn,verified,verified,admin_certified,2019-12-10 15:41:04.976560,4
3,2446861,37919,AP8138631663,67,stopped,None,usdt,7.060000,ngn,"5,853.000000",nan,2022-03-30 21:20:29.729608,2022-03-31 22:23:33.396051,2022-03-31 22:23:36.178010,2022-03-30 21:20:29.699266,2022-03-31 22:23:36.431936,7.036751,"5,853.000000",7.263555,"5,720.092915",7.266024,"5,721.404320",0.002469,1.311405,"4,871.220938",62731,64460,0.300000,True,None,active,vn,verified,verified,peer_certify_pending,2020-07-22 09:42:28.122860,4
4,1441909,2899,AP3837074128,67,aborted,slippage_tolerance_exceeded,usdt,11.700000,ngn,"6,288.000000",nan,2021-12-09 12:44:29.803576,NaT,NaT,2021-12-09 12:44:29.398231,2021-12-09 12:44:34.298315,nan,nan,nan,nan,nan,nan,nan,nan,nan,63282,63546,0.050000,None,"amount1 < amount1Min, amount1=5899698156048218...",active,vn,verified,verified,admin_certified,2019-09-25 12:42:02.354132,4


In [8]:
def calculate_amount_x(L, pa, pb, P):
    if P <= pa:
        return L * (np.power(np.sqrt(pa), -1) - np.power(np.sqrt(pb), -1))
    
    if P >= pb:
        return 0
    
    return L * (np.power(np.sqrt(P), -1) - np.power(np.sqrt(pb), -1))
    
def calculate_amount_y(L, pa, pb, P):
    if P <= pa:
        return 0
    
    if P >= pb:
        return L * (np.sqrt(pb) - np.sqrt(pa))
    
    return L * (np.sqrt(P) - np.sqrt(pa))

In [9]:
if PRICE_TYPE == 0:
    current_price = np.float64(CURRENT_PRICE)
else:
    current_price = np.float64(1.0) / np.float64(CURRENT_PRICE)
    
df_positions = df_positions[df_positions['status'] != "aborted"].reset_index(drop=True)
df_positions['current_price'] = current_price
df_positions['price_lower_bound'] = np.power(1.0001, df_positions['tick_lower'])
df_positions['price_upper_bound'] = np.power(1.0001, df_positions['tick_upper'])

def calc_current_amount_token0(row):
    return calculate_amount_x(row['liquidity'], row['price_lower_bound'], row['price_upper_bound'], row['current_price'])
    
def calc_current_amount_token1(row):
    return calculate_amount_y(row['liquidity'], row['price_lower_bound'], row['price_upper_bound'], row['current_price'])

df_positions['amount_token0_current'] = df_positions.apply(calc_current_amount_token0, axis=1)
df_positions['amount_token1_current'] = df_positions.apply(calc_current_amount_token1, axis=1)

In [10]:
def calculate_total_asset_in_tick(tick, tick_next):
# tick = 99656 # row['tick']
# tick_next = 101763 # row['next_tick']

    positions_in_tick = df_positions[(df_positions['tick_lower'] <= tick) & (df_positions['tick_upper'] >= tick_next)].copy().reset_index(drop=True)

    positions_in_tick['current_tick'] = tick
    positions_in_tick['next_tick'] = tick_next
    
    positions_in_tick['pa'] = np.power(1.0001, positions_in_tick['current_tick'])
    positions_in_tick['pb'] = np.power(1.0001, positions_in_tick['next_tick'])
    
    positions_in_tick['amount_token0_in_tick'] = positions_in_tick.apply(lambda row: calculate_amount_x(row['liquidity'], row['pa'], row['pb'], row['current_price']), axis=1)
    positions_in_tick['amount_token1_in_tick'] = positions_in_tick.apply(lambda row: calculate_amount_y(row['liquidity'], row['pa'], row['pb'], row['current_price']), axis=1)

    value = positions_in_tick \
        .groupby(['current_tick', 'next_tick']) \
        .agg(total_amount_token0_in_tick=('amount_token0_in_tick', 'sum'),
             total_amount_token1_in_tick=('amount_token1_in_tick', 'sum')) \
        .reset_index()

    return value.loc[0, 'total_amount_token0_in_tick'], value.loc[0, 'total_amount_token1_in_tick']

In [11]:
def get_amount_token0_in_tick(row):
    amount0, amount1 = calculate_total_asset_in_tick(row['tick'], row['next_tick'])
    return amount0

def get_amount_token1_in_tick(row):
    amount0, amount1 = calculate_total_asset_in_tick(row['tick'], row['next_tick'])
    return amount1

list_ticks = list(sorted(set(df_positions['tick_lower'].tolist() + df_positions['tick_upper'].tolist())))
df_ticks = pd.DataFrame(list_ticks, columns=['tick'])
df_ticks['next_tick'] = df_ticks['tick'].shift(-1)
df_ticks=df_ticks[~df_ticks['next_tick'].isna()]
df_ticks['next_tick'] = df_ticks['next_tick'].astype(int)

df_ticks['price'] = np.power(1.0001, df_ticks['tick'])
df_ticks['next_price'] = np.power(1.0001, df_ticks['next_tick'])
df_ticks['current_price'] = current_price

df_ticks['total_amount_token0'] = df_ticks.apply(get_amount_token0_in_tick, axis=1)
df_ticks['total_amount_token1'] = df_ticks.apply(get_amount_token1_in_tick, axis=1)

df_ticks['liquidity0'] = np.where(df_ticks['total_amount_token0'] > 0, df_ticks['total_amount_token0'] * np.sqrt(df_ticks['current_price']) * np.sqrt(df_ticks['next_price']) / (np.sqrt(df_ticks['next_price']) - np.sqrt(df_ticks['current_price'])), 0)
df_ticks['liquidity1'] = np.where(df_ticks['total_amount_token1'] > 0, df_ticks['total_amount_token1'] / (np.sqrt(df_ticks['current_price']) - np.sqrt(df_ticks['price'])), 0)
df_ticks['liquidity'] = np.where(df_ticks['liquidity0'] > 0, df_ticks['liquidity0'], df_ticks['liquidity1'])

df_ticks.head()

,tick,next_tick,price,next_price,current_price,total_amount_token0,total_amount_token1,liquidity0,liquidity1,liquidity
0,-120811,-120669,0.000006,0.000006,582.000000,0.000000,0.004443,0.000000,0.000184,0.000184
1,-120669,-120575,0.000006,0.000006,582.000000,0.000000,0.004985,0.000000,0.000207,0.000207
2,-120575,-120565,0.000006,0.000006,582.000000,0.000000,0.000706,0.000000,0.000029,0.000029
3,-120565,-120558,0.000006,0.000006,582.000000,0.000000,0.000901,0.000000,0.000037,0.000037
4,-120558,47229,0.000006,112.467453,582.000000,0.000000,"457,452.920694",0.000000,"18,963.929086","18,963.929086"


In [12]:
target_price = current_price * (1.0 + TARGET_SPLIPPAGE)
target_tick = int(np.ceil(np.log(target_price) / np.log(1.0001)))
current_tick = int(np.floor(np.log(current_price) / np.log(1.0001)))

print(f'Current price: {current_price}, nearest tick: {current_tick}')
print(f'Target price increase: {target_price}, nearest tick: {target_tick}')
print('-' * 50)

delta_token0 = 0
delta_token0_in_pool = 0

for index, row in df_ticks.iterrows():
    if row['next_tick'] <= current_tick:
        continue
    
    if row['tick'] >= target_tick:
        break
    
    liquidity = row['liquidity']
    token0_in_pool = row['total_amount_token0']
    token1_in_pool = row['total_amount_token1']
    
    tick_lower_in_pool = int(row['tick'])
    tick_upper_in_pool = int(row['next_tick'])
    
    price_for_tick_lower = row['price']
    price_for_tick_upper = row['next_price']
    
    if target_tick >= tick_upper_in_pool:
        delta_token0_in_pool = token0_in_pool
    else:
        delta_token0_in_pool = calculate_amount_y(liquidity, price_for_tick_lower, price_for_tick_upper, target_tick)
            
    delta_token0 += delta_token0_in_pool
            
    print(f'Pool tick {tick_lower_in_pool}-{tick_upper_in_pool}, price range {price_for_tick_lower}-{price_for_tick_upper}, total token0 get in this pool {delta_token0_in_pool}, total token0 get {delta_token0}')

Current price: 582.0, nearest tick: 63667
Target price increase: 587.82, nearest tick: 63768
--------------------------------------------------
Pool tick 63667-63680, price range 581.9483430929733-582.7053300251806, total token0 get in this pool 14612.970384571083, total token0 get 14612.970384571083
Pool tick 63680-63685, price range 582.7053300251806-582.9967409665535, total token0 get in this pool 27244.366638325773, total token0 get 41857.337022896856
Pool tick 63685-63691, price range 582.9967409665535-583.3466264723053, total token0 get in this pool 35935.25684150441, total token0 get 77792.59386440126
Pool tick 63691-63702, price range 583.3466264723053-583.9886286983408, total token0 get in this pool 103250.35758000337, total token0 get 181042.95144440464
Pool tick 63702-63704, price range 583.9886286983408-584.1054322639667, total token0 get in this pool 18906.35347037747, total token0 get 199949.30491478211
Pool tick 63704-63706, price range 584.1054322639667-584.222259191473

In [13]:
target_price = current_price * (1.0 - TARGET_SPLIPPAGE)
target_tick = int(np.floor(np.log(target_price) / np.log(1.0001)))
current_tick = int(np.ceil(np.log(current_price) / np.log(1.0001)))

print(f'Current price: {current_price}, nearest tick: {current_tick}')
print(f'Target price increase: {target_price}, nearest tick: {target_tick}')
print('-' * 50)

delta_token1 = 0
delta_token1_in_pool = 0

for index, row in df_ticks[::-1].iterrows():
    if row['tick'] >= current_tick:
        continue
    
    if row['next_tick'] <= target_tick:
        break
    
    liquidity = row['liquidity']
    token0_in_pool = row['total_amount_token0']
    token1_in_pool = row['total_amount_token1']
    
    tick_lower_in_pool = int(row['tick'])
    tick_upper_in_pool = int(row['next_tick'])
    
    price_for_tick_lower = row['price']
    price_for_tick_upper = row['next_price']
    
    if target_tick <= tick_lower_in_pool:
        delta_token1_in_pool = token1_in_pool
    else:
        delta_token1_in_pool = calculate_amount_x(liquidity, price_for_tick_lower, price_for_tick_upper, target_tick)
            
    delta_token1 += delta_token1_in_pool
            
    print(f'Pool tick {tick_lower_in_pool}-{tick_upper_in_pool}, price range {price_for_tick_lower}-{price_for_tick_upper}, total token0 get in this pool {delta_token1_in_pool}, total token0 get {delta_token1}')

Current price: 582.0, nearest tick: 63668
Target price increase: 576.18, nearest tick: 63567
--------------------------------------------------
Pool tick 63667-63680, price range 581.9483430929733-582.7053300251806, total token0 get in this pool 623450.069371003, total token0 get 623450.069371003
Pool tick 63650-63667, price range 580.9599207270542-581.9483430929733, total token0 get in this pool 10482134.354161486, total token0 get 11105584.42353249
Pool tick 63633-63650, price range 579.9731771678282-580.9599207270542, total token0 get in this pool 7327190.658490626, total token0 get 18432775.082023114
Pool tick 63598-63633, price range 577.9469203766433-579.9731771678282, total token0 get in this pool 12740072.137142727, total token0 get 31172847.21916584
Pool tick 63581-63598, price range 576.9652943110407-577.9469203766433, total token0 get in this pool 6071444.078121479, total token0 get 37244291.297287315
Pool tick 63564-63581, price range 575.9853355088123-576.9652943110407, to

In [14]:
print('SUMMARY')
print(f'Pool {TOKEN0}/{TOKEN1} at {DATETIME_REPORT}')
print(f'Current price: {current_price}')
print(f'Slippage {TARGET_SPLIPPAGE*100}% increase at price {current_price * (1.0 + TARGET_SPLIPPAGE)}, total token get: {delta_token0:,} {TOKEN0}')
print(f'Slippage {TARGET_SPLIPPAGE*100}% decrease at price {current_price * (1.0 - TARGET_SPLIPPAGE)}, total token get: {delta_token1:,} {TOKEN1}')

SUMMARY
Pool usdt/ngn at 2022-03-31 12:00:00
Current price: 582.0
Slippage 1.0% increase at price 587.82, total token get: 1,608,816.9600915234 usdt
Slippage 1.0% decrease at price 576.18, total token get: 37,244,291.297287315 ngn
